In [1]:
import numpy as np
import pandas as pd


In [2]:
#Units of the table
#wind_speed = m/s
#air_temp = degree C
#rel_hum = unitless
#net_rad = W/m2
#Rainfall = mm
#Runoff = mm
gamma = 0.063875 #kPa/C
#ke = 0.0013 #(s/d)(1/kPa)
ke = 1.26/1000 #1/kPa
df = pd.read_csv('Evapotranspiration.csv')

In [3]:
df

,Month,wind_speed,air_temp,rel_hum,net_rad,rainfall,runoff
0,Jan,2.00,4,0.33,88,10.9,1.0
1,Feb,1.10,6,0.43,95,15.5,2.0
2,March,2.20,8,0.50,110,31.8,4.4
3,Apr,3.00,12,0.55,135,53.8,5.0
4,May,1.80,14,0.63,155,88.1,7.0
5,Jun,1.50,20,0.65,180,91.4,11.0
6,Jul,1.00,23,0.70,210,77.5,6.0
7,Aug,0.80,25,0.75,205,66.3,6.7
8,Sep,0.75,23,0.73,175,45.2,5.1
9,Oct,1.10,18,0.68,100,33.3,5.0


In [4]:
df['es'] = 0.611*np.exp((17.27*df['air_temp'])/(237.3+df['air_temp'])) #calculating saturated Vapor pressure #kPa
df['ea'] = df['es']*df['rel_hum'] #converting relative humidity to percent #kPa

In [5]:
#Assuming surface temperature smae as sair tempereature
df['delta'] = (4098*df['es'])/np.power((237.3+df['air_temp']),2)

In [6]:
#Need to convert wind speed to mm/month
#I assume 30 days in each month, to make calculation simpler
#df['wind_speed'] = df['wind_speed']*60*60*24*1000 #mm/month
#Now Use Penman to calculate evaporation from lake surface
#Assume Ground Heat Flux to be zero
df['evap'] = (df['delta']*df['net_rad'] + ke*gamma*28.34*df['wind_speed']*(df['es']-df['ea']))/(28.34*(df['delta']+gamma))

In [7]:
df

,Month,wind_speed,air_temp,rel_hum,net_rad,rainfall,runoff,es,ea,delta,evap
0,Jan,2.00,4,0.33,88,10.9,1.0,0.813527,0.268464,0.057257,1.468477
1,Feb,1.10,6,0.43,95,15.5,2.0,0.935416,0.402229,0.064758,1.687946
2,March,2.20,8,0.50,110,31.8,4.4,1.073120,0.536560,0.073084,2.071912
3,Apr,3.00,12,0.55,135,53.8,5.0,1.403023,0.771663,0.092511,2.818897
4,May,1.80,14,0.63,155,88.1,7.0,1.599128,1.007451,0.103770,3.385931
5,Jun,1.50,20,0.65,180,91.4,11.0,2.339047,1.520380,0.144788,4.407640
6,Jul,1.00,23,0.70,210,77.5,6.0,2.810358,1.967250,0.169975,5.386300
7,Aug,0.80,25,0.75,205,66.3,6.7,3.168815,2.376611,0.188744,5.404769
8,Sep,0.75,23,0.73,175,45.2,5.1,2.810358,2.051561,0.169975,4.488538
9,Oct,1.10,18,0.68,100,33.3,5.0,2.064665,1.403972,0.129814,2.365221


In [8]:
#orignal Penman
#df['evap_orig'] = (df['delta']/(df['delta']+gamma))*(df['net_rad']/28.34)+((gamma*(0.0026*(1+0.54*df))))